# <키워드, 날짜> 기준 네이버 뉴스 크롤러

# Ctrl + Enter 여러번 누르거나
# Ctrl + F9 한번 누르기

In [1]:
!pip install BeautifulSoup
!pip install openpyxl

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\bangm\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\bangm\\AppData\\Local\\Temp\\pip-install-tmpji2pe\\beautifulsoup_0922483021fa4417b267551a7c8b310a\\setup.py'"'"'; __file__='"'"'C:\\Users\\bangm\\AppData\\Local\\Temp\\pip-install-tmpji2pe\\beautifulsoup_0922483021fa4417b267551a7c8b310a\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\bangm\AppData\Local\Temp\pip-pip-egg-info-gbp45ldd'
         cwd: C:\Users\bangm\AppData\Local\Temp\pip-install-tmpji2pe\beautifulsoup_0922483021fa4417b267551a7c8b310a\
    Complete output (6 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module

In [2]:
print("KEYWORD: ")
my_keyword = input()
print("START_DATE(YYYY.MM.DD): ")
my_start_date1 = input()
my_start_date2 = my_start_date1.replace('.','')
print("END_DATE(YYYY.MM.DD): ")
my_end_date1 = input()
my_end_date2 = my_end_date1.replace('.','')
print("NUMBER: ")
my_number = int(input())

KEYWORD: 
서강대
START_DATE(YYYY.MM.DD): 
2022.10.01
END_DATE(YYYY.MM.DD): 
2022.10.15
NUMBER: 
100


### 명령어 입력

### 완성코드

In [3]:
import requests
from bs4 import BeautifulSoup

num_list =[]

for num in range(0,my_number,10):
    num_list.append(num)

In [4]:
articles = []

for number in num_list:
    URL = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_word}&sort=0&photo=0&field=0&pd=3&ds={startdate}&de={enddate}&cluster_rank=37&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{start_date}to{end_date},a:all&start={page_num}'
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
        'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
    url = URL.format(search_word = my_keyword ,startdate= my_start_date1,enddate = my_end_date1,
                                   start_date = my_start_date2,end_date = my_end_date2 ,page_num = number)
    
    response = requests.get(url, headers = headers)
    soup=BeautifulSoup(response.text,'lxml')
    news_area = soup.find_all('div', {'class' : 'news_area'})
    articles += news_area

In [5]:
dates = []
presses = []
titles = []
summs = []
links = []

for article in articles:
    article_link = article.find('a', attrs = {'class' : 'news_tit'})['href']
    article_title = article.find('a', attrs = {'class' : 'news_tit'})['title']
    article_summ = article.find('a', attrs = {'class' : 'api_txt_lines dsc_txt_wrap'}).get_text()
    date2 = article.find('span', attrs = {'class' : 'info'}).get_text()
    press2 = article.find('a', {'class': 'info press'}).get_text()
    press2 = press2.replace("언론사 선정",'')
    dates.append(date2)
    presses.append(press2)
    titles.append(article_title)
    summs.append(article_summ)
    links.append(article_link)

In [6]:
import pandas as pd
df = pd.DataFrame({'날짜': dates,
                   '언론사': presses,
                   '제목': titles,
                   '요약': summs,
                   '링크' : links})

In [7]:
df.to_excel('rawdata.xlsx')

In [8]:
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
wb1 = openpyxl.load_workbook('rawdata.xlsx', data_only=True)
excel_sheet = wb1.active

In [9]:
wb1.save(my_keyword+'_'+my_start_date1+"_to_"+my_end_date1+'('+str(len(df))+'개)'+'.xlsx')